# Descripción del proyecto
Lo has hecho de maravilla en el curso de TripleTen y te han ofrecido hacer prácticas en el departamento de analítica de Showz, una empresa de venta de entradas de eventos. Tu primera tarea es ayudar a optimizar los gastos de marketing. 

Cuentas con:

- registros del servidor con datos sobre las visitas a Showz desde enero de 2017 hasta diciembre de 2018;
- un archivo con los pedidos en este periodo;
- estadísticas de gastos de marketing.

Lo que vas a investigar: 

- cómo los clientes usan el servicio;
- cuándo empiezan a comprar;
- cuánto dinero aporta cada cliente a la compañía;
- cuándo los ingresos cubren el costo de adquisición de los clientes.

## Paso 1. Acceda los datos y prepáralos para el análisis
Almacena los datos de visitas, pedidos y gastos en variables. Optimiza los datos para el análisis. Asegúrate de que cada columna contenga el tipo de datos correcto. Rutas de archivos:

- /datasets/visits_log_us.csv (registros del servidor con datos sobre las visitas al sitio web)
- /datasets/orders_log_us.csv (datos sobre pedidos)
- /datasets/costs_us.csv (datos sobre gastos de marketing)


In [28]:
# Importamos las librerias que podamos ocupar y las bases de datos.

import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt

visits = pd.read_csv('archives/visits_log_us.csv')
orders = pd.read_csv('archives/orders_log_us.csv')
costs_mkt = pd.read_csv('archives/costs_us.csv')

Vamos a explorar el archivo vosots_log_us y revisar sus tipos de datos para el DF visit.

In [29]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB


- No tenemos valores faltantes, repetidos muy probablemte si pero esos los quitamos especificamente en los análisis posteriores,
- convertimos los nombres de las columnas a snake_case en minúsculas y
- cambiamos el Dtype de las fechas a .datetime ya que las necesitaremos para los análisis más adelante.

In [30]:
visits.columns = visits.columns.str.lower().str.replace(' ', '_')

In [31]:
visits['start_ts'] = pd.to_datetime(visits['start_ts'], format= "%Y-%m-%d %H:%M:%S")
visits['end_ts'] = pd.to_datetime(visits['end_ts'], format= "%Y-%m-%d %H:%M:%S")

visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   device     359400 non-null  object        
 1   end_ts     359400 non-null  datetime64[ns]
 2   source_id  359400 non-null  int64         
 3   start_ts   359400 non-null  datetime64[ns]
 4   uid        359400 non-null  uint64        
dtypes: datetime64[ns](2), int64(1), object(1), uint64(1)
memory usage: 13.7+ MB


Ahora vamos a explorar y analizar el archivo order_log_us del DF orders

In [32]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB


- No tenemos valores faltantes, repetidos muy probablemte si pero esos los quitamos especificamente en los análisis posteriores,
- convertimos los nombres de las columnas a snake_case en minúsculas y
- cambiamos el Dtype de las fechas a .datetime ya que las necesitaremos para los análisis más adelante.

In [33]:
orders.columns = orders.columns.str.lower().str.replace(' ', '_')

In [34]:
orders['buy_ts'] = pd.to_datetime(orders['buy_ts'])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   buy_ts   50415 non-null  datetime64[ns]
 1   revenue  50415 non-null  float64       
 2   uid      50415 non-null  uint64        
dtypes: datetime64[ns](1), float64(1), uint64(1)
memory usage: 1.2 MB


Por ultimo, analizaremos el archivo costs_us.csv del DF costs_mkt.

In [35]:
costs_mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   source_id  2542 non-null   int64  
 1   dt         2542 non-null   object 
 2   costs      2542 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.7+ KB


- No tenemos valores faltantes, repetidos muy probablemte si pero esos los quitamos especificamente en los análisis posteriores, solo
- cambiamos el Dtype de las fechas a .datetime ya que las necesitaremos para los análisis más adelante.

In [36]:
costs_mkt['dt'] = pd.to_datetime(costs_mkt['dt'])

costs_mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   source_id  2542 non-null   int64         
 1   dt         2542 non-null   datetime64[ns]
 2   costs      2542 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 59.7 KB


## Paso 2. Haz informes y calcula métricas 

### 1. Visitas:
- 1.1 ¿Cuántas personas lo usan cada día, semana y mes?
- 1.2 ¿Cuántas sesiones hay por día? (Un usuario puede tener más de una sesión).
- 1.3 ¿Cuál es la duración de cada sesión?
- 1.4 ¿Con qué frecuencia los usuarios regresan?

Vamos empezar imprimiendo las primeras 5 filas de la tabla para generar una noción con lo que estamos trabajando.

In [37]:
visits.head(5)

,device,end_ts,source_id,start_ts,uid
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168


Crearemos columnas separadas para los valores de año, mes, semana y fecha que utilizaremos para hacer el análisis de las preguntas siguientes

In [38]:
# Separación de tipo de datos, para año y semana es necesario utilizar isocalendar().
visits['session_year'] = visits['start_ts'].dt.isocalendar().year
visits['session_month'] = visits['start_ts'].dt.month
visits['session_week'] = visits['start_ts'].dt.isocalendar().week
visits['session_date'] = visits['start_ts'].dt.date

visits.head(5)

,device,end_ts,source_id,start_ts,uid,session_year,session_month,session_week,session_date
0,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,16879256277535980062,2017,12,51,2017-12-20
1,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00,104060357244891740,2018,2,8,2018-02-19
2,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00,7459035603376831527,2017,7,26,2017-07-01
3,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00,16174680259334210214,2018,5,20,2018-05-20
4,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00,9969694820036681168,2017,12,52,2017-12-27


### 1.1 ¿Cuántas personas lo usan cada día, semana y mes?

Utilizaremos las métricas de DAU, WAU y MAU para saber cuántos usuarios lo usan. 

In [39]:
dau = visits.groupby('session_date').agg({'uid': 'nunique'}).mean().astype('int64')
wau = visits.groupby(['session_year', 'session_week']).agg({'uid': 'nunique'}).mean().astype('int64')
mau = visits.groupby(['session_year', 'session_month']).agg({'uid': 'nunique'}).mean().astype('int64')

print(f"El DAU total es: {dau}")
print(f"El WAU total es: {wau}")
print(f"El MAU total es: {mau}")

El DAU total es: uid    907
dtype: int64
El WAU total es: uid    5716
dtype: int64
El MAU total es: uid    23228
dtype: int64


### Conclusión:
- 907 pesronas en promedio visitan la página por día,
- 5,716 personas en promedio visitan la página por semana y
- 23,228 personas en promedio por mes. 

### 1.2 ¿Cuántas sesiones hay por día?

In [40]:
sessions_per_day = visits.groupby('session_date').agg({'uid': 'count'}).mean().astype('int64')
session_every_day = visits.groupby('session_date').agg({'uid': 'count'}).astype('int64')

print(f"Sesiones por día: {sessions_per_day}")
print(session_every_day)

Sesiones por día: uid    987
dtype: int64
               uid
session_date      
2017-06-01     664
2017-06-02     658
2017-06-03     477
2017-06-04     510
2017-06-05     893
...            ...
2018-05-27     672
2018-05-28    1156
2018-05-29    1035
2018-05-30    1410
2018-05-31    2256

[364 rows x 1 columns]


### Conclusión:
Hay en promedio unas 987 sesiones por día, hay que considerar que los usuarios pueden realizar más de una sesión diaria.

### 1.3 ¿Cuál es la duración de cada sesión?

Aplicaremos una resta de la columna del tiempo que termina la sessión ('end_ts') menos el tiempo que inicia la sesión. 

In [41]:
visits['duration_session'] = (visits['end_ts'] - visits['start_ts'])


duration_mean = visits['duration_session'].mean()

print(f"Duración promedio de usuarios: {duration_mean}")
print(visits['duration_session'])

Duración promedio de usuarios: 0 days 00:10:43.025687256
0        0 days 00:18:00
1        0 days 00:28:00
2        0 days 00:00:00
3        0 days 00:24:00
4        0 days 00:00:00
               ...      
359395   0 days 00:00:19
359396   0 days 00:00:19
359397   0 days 00:00:19
359398   0 days 00:00:19
359399   0 days 00:00:19
Name: duration_session, Length: 359400, dtype: timedelta64[ns]


### Conclusión:
La duración promedio de los usuarios por sesión es de 11 minútos.

### 1.4 ¿Con qué frecuencia los usuarios regresan?

Utilizaremos el sticky factor para calcular la frecuencia de retorno de los usuarios.

In [44]:
# sticky factor mensual 
sticky_factor_month = (dau / mau) * 100
# sticky factor semanal 
sticky_factor_week = (dau / wau) * 100

print(f"El sticky factor mensual es de: {sticky_factor_month}")
print(f"El sticky factor semanal es de: {sticky_factor_week}")

El sticky factor mensual es de: uid    3.90477
dtype: float64
El sticky factor semanal es de: uid    15.86774
dtype: float64


### Conclusión:
- La frecuencia con la que regresan los usuarios semanalmente es del 15.8 porciento, y 
- la frecuencia con que regresan mensualmente es del 3.9 porciento.